# Visualizations

## Setup

In [ ]:
import matplotlib.pyplot as plt
from herbie import FastHerbie, paint, wgrib2, Herbie
# from herbie.toolbox import EasyMap, ccrs, pc
import xarray as xr
from datetime import datetime
import sys
import os.path as osp
import pandas as pd
import numpy as np
sys.path.append("../src")
from utils import Dict, read_yml, str2time, print_dict_summary, read_pkl
from viz import map_var, create_gif

In [ ]:
with open("../etc/forecast_config.json", "r") as json_file:
    config = json.load(json_file)   
    config = Dict(config)
config.update({
    'start_time': "2024-12-01T00:00:00Z",
    'end_time': "2024-12-02T23:00:00Z"
})
print_dict_summary(config)

In [ ]:
## Retrieve Data
# hrrr_ds = ih.retrieve_hrrr(config)
# hrrr_ds["date_time"] = hrrr_ds["date_time"].astype("datetime64[ns]")

## Save Data
# hrrr_ds.to_netcdf("../data/hrrr_ds.nc")

## Read HRRR Data
hrrr_ds = xr.open_dataset("../data/hrrr_ds.nc")

In [ ]:
config.update({
    'start_time': "2023-12-01T00:00:00Z",
    'end_time': "2023-12-07T23:00:00Z"
})
raws_dict = rrs.build_raws_dict(config)

## Viz

### Animated Maps

In [ ]:
H = Herbie("2024-01-01", product="sfc")
ds = H.xarray("(?:HGT|LAND):surface")

hrrr_ds["Ew"] = hrrr_ds.Ew.where(ds.lsm > 0)

In [ ]:
mt_time = pd.to_datetime(hrrr_ds.valid_time.values).tz_localize("UTC").tz_convert("America/Denver")

In [ ]:
tstep = 0

t = hrrr_ds.valid_time[tstep]
t = f"{t.dt.year.item():04d}-{t.dt.month.item():02d}-{t.dt.day.item():02d} {t.dt.hour.item():02d}:{t.dt.minute.item():02d}:{t.dt.second.item():02d}"

map_var(hrrr_ds, "Ew", 
        time_step = tstep,
        legend_title = "Fuel Moisture Content (%)",
        title = f"FMC Forecast at {t} UTC"
       )

In [ ]:
# Define time steps (e.g., first 10)
time_steps = range(0, 48)  # Adjust as needed
create_gif(hrrr_ds, "Ew", time_steps, legend_title="Fuel Moisture Content (%)", gif_path='../outputs/fmc.gif', duration=0.5)


In [ ]:
# create_gif(hrrr_ds, "rh", time_steps, legend_title="Relative Humidity (%)", gif_path='../outputs/rh.gif', duration=0.5)

In [ ]:
# create_gif(hrrr_ds, "temp", time_steps, legend_title="Air Temperature (K)", gif_path='../outputs/temp.gif', duration=0.5)

In [ ]:
# create_gif(hrrr_ds, "wind", time_steps, legend_title="Wind Speed 10m (m/s)", gif_path='../outputs/wind.gif', duration=0.5)

### Timeseries Plots

In [ ]:
raws_dict.keys()

In [ ]:
st = "HSYN1"
y = raws_dict[st]["RAWS"]["fm"].to_numpy()
x = raws_dict[st]["times"]
plt.plot(y, linestyle='-',c='#468a29',label='FM Observed')
plt.xlabel("Hour")
plt.ylabel("FMC (%)")
plt.savefig("../outputs/ts.png")

In [ ]:
st = "TT562"
y = raws_dict[st]["RAWS"]["fm"].to_numpy()
x = raws_dict[st]["times"]
plt.plot(x, y, linestyle='-',c='#468a29',label='FM Observed')
plt.legend()
plt.title(f"Observed FMC at RAWS {st}")
plt.xticks(rotation=90)
plt.savefig("../outputs/fmc_ts.png")